In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('africa_merged_dataset.csv')
solar = pd.read_csv('../../energy_strength/solar/solar_countries.csv')
# solar = solar[solar['iso_a3'].isin(list(df['iso3_code']))]
wind = pd.read_csv('../../energy_strength/wind/wind_countries.csv')
# wind = wind[wind['iso_a3'].isin(list(df['iso3_code']))]


In [ ]:
df.sample(5)
# get rows count of df
# print(df.shape[0])

In [ ]:
solar.sample(5)

In [ ]:
wind.sample(5)

In [3]:
# df.drop(['wind_speed', 'solar'], axis=1, inplace=True)
import numpy as np
from scipy.spatial import distance
def generateDatasetNodeList(datasetDF, targetVar):
    nodes = []
    nodeWVar = []
    for index, row in datasetDF.iterrows():
        nodes.append((row['lat'], row['lon']))
        nodeWVar.append((row['lat'], row['lon'], row[targetVar]))
    return np.asarray(nodes), nodeWVar

solarNodes, solarNodesWVar = generateDatasetNodeList(solar, 'ALLSKY_SFC_SW_DWN')
windNodes, windNodesWVar = generateDatasetNodeList(wind, 'WS50M')

len(solarNodes)
len(windNodes)

df.shape[0]


    # return np.argmin(distVals, axis=1), np.min(distVals, axis=1)
# def closest_node(node, nodes):
#     distVals = distance.cdist([node], nodes)
#     minIdx = distVals.argmin()
#     # closest_index = distance.cdist([node], nodes).argmin()
#     return minIdx, distVals[0][minIdx]

solarThreshold = distance.euclidean((0, 0), (1, 1)) * 1.5
windThreshold = distance.euclidean((0, 0), (0.5, 0.625)) * 1.5

# import cProfile

# pr = cProfile.Profile()
# iterate through df and for each row, replace wind_speed and solar with 0
# pr.enable()
maxDistSolar = 0
maxDistWind = 0
maxDistSolarIdx = -1
maxDistWindIdx = -1
finalList = []
dfList = []
for index, row in df.iterrows():
    if index % 10000 == 0 and index != 0:
        print(index)
        # pr.disable()
        # import pstats, io
        # from pstats import SortKey
        # s = io.StringIO()
        # sortby = SortKey.CUMULATIVE
        # ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
        # ps.print_stats()
        # print(s.getvalue())
        # break
    dfList.append((row['lat'], row['long']))


    # closestIdx, dist = closest_node((row['lat'], row['long']), solarNodes)
    # solarVal = solarNodesWVar[closestIdx][2]

    # if dist > maxDistSolar:
    #     maxDistSolar = dist
    #     maxDistSolarIdx = ((row['lat'], row['long']), solarNodesWVar[closestIdx])

    # # if dist < solarThreshold:
    # #     solarVal = solarNodesWVar[closestIdx][2]
    # # else:
    # #     solarVal = np.NaN

    # closestIdx, dist = closest_node((row['lat'], row['long']), windNodes)
    # windVal = windNodesWVar[closestIdx][2]

    # if dist > maxDistWind:
    #     maxDistWind = dist
    #     maxDistWindIdx = ((row['lat'], row['long']), windNodesWVar[closestIdx])
    # # if dist < windThreshold:
    # #     windVal = windNodesWVar[closestIdx][2]
    # # else:
    # #     windVal = np.NaN

    # finalList.append([row['lat'], row['long'], solarVal, windVal])
    # df.loc[index, 'solar'] = 0
    # df.loc[index, 'wind_speed'] = 0
# (maxDistWind, maxDistWindIdx, maxDistSolar, maxDistSolarIdx)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000


In [4]:
print(len(solarNodes))
print(len(windNodes))

df.shape[0]

64800
207986


625522

In [5]:
dfListNPArr = np.asarray(dfList)

def closest_node(nodes, nodeList):
    retVals = []
    i = 0
    # i = 625000
    while True:
        if i % 10000 == 0:
            print(i)
        if i + 1000 > len(nodes):
            distVals = distance.cdist(nodes[i:], nodeList, "sqeuclidean")
            retVals += np.argmin(distVals, axis=1).tolist()
            break
        distVals = distance.cdist(nodes[i:i+1000], nodeList, "sqeuclidean")
        retVals += np.argmin(distVals, axis=1).tolist()
        i += 1000
    return retVals
    
    
    # return np.argmin(distVals, axis=1)


finalList = []

windClosest = closest_node(dfListNPArr, windNodes)
solarClosest = closest_node(dfListNPArr, solarNodes)




0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000


In [6]:
print(len(solarNodes))
print(len(windNodes))
print(len(solarClosest))
print(len(windClosest))
print(len(dfList))

64800
207986
625522
625522
625522


In [10]:
finalList = []

for i in range(len(dfList)):
    if index % 10000 == 0 and index != 0:
        print(index)
    
    solarVal = solarNodesWVar[solarClosest[i]][2]
    windVal = windNodesWVar[windClosest[i]][2]
    finalList.append([dfList[i][0], dfList[i][1], solarVal, windVal])

print(len(finalList))

625522


In [12]:
solarWindDF = pd.DataFrame(finalList, columns=['lat', 'long', 'solar', 'wind_speed'])

# count number of NaNs in solar and wind_speed columns
# print(solarWindDF['solar'].isna().sum())
# print(solarWindDF['wind_speed'].isna().sum())

# print first 5 rows with NaNs
# print(solarWindDF[solarWindDF['solar'].isna()].head())

# # drop solar and wind_speed columns from df
df.drop(['wind_speed', 'solar'], axis=1, inplace=True)

# # merge df and solarWindDF on lat and long
df = df.merge(solarWindDF, on=['lat', 'long'], how='left')

# # write df to a csv
df.to_csv('africa_merged_dataset_nasa_power.csv', index=False)